In [1]:
# How to use Pretrained models with PyTorch
# Simple Classifier using resnet50
# code by GunhoChoi

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.models as models
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.autograd import Variable
from torchvision.models.vgg import model_urls
from torchviz import make_dot
import os

batch_size = 3
learning_rate =0.0002
epoch = 50

resnet = models.resnet50(pretrained=True)
# make_dot(resnet)

In [2]:
# Input pipeline from a folder containing multiple folders of images
# we can check the classes, class_to_idx, and filename with idx

img_dir = "./images"
img_data = dset.ImageFolder(img_dir, transforms.Compose([
            transforms.RandomSizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            ]))

print(img_data.classes)
print(img_data.class_to_idx)
print(img_data.imgs)

['dogs', 'onepun']
{'onepun': 1, 'dogs': 0}
[('./images/onepun/5720.jpg', 1), ('./images/onepun/1454.jpg', 1), ('./images/onepun/5320.jpg', 1), ('./images/dogs/3b1a8d3fca2adb68a7a908b73005764a.jpg', 0), ('./images/dogs/dog-650299.jpg', 0)]


/usr/local/lib/python2.7/dist-packages/torchvision-0.1.8-py2.7.egg/torchvision/transforms/transforms.py:563: UserWarning: The use of the transforms.RandomSizedCrop transform is deprecated, please use transforms.RandomResizedCrop instead.


In [3]:
# After we get the list of images, we can turn the list into batches of images
# with torch.utils.data.DataLoader()

img_batch = data.DataLoader(img_data, batch_size=batch_size,
                            shuffle=True, num_workers=2)

for img,label in img_batch:
    print(img.size())
    print(label)

torch.Size([3, 3, 224, 224])
tensor([1, 0, 1])
torch.Size([2, 3, 224, 224])
tensor([0, 1])


In [4]:
# test of the result coming from resnet model

img = Variable(img)
print(resnet(img))
print(resnet(img).shape)

tensor([[-0.6667, -0.9177, -2.1880,  ...,  0.6528,  1.7321,  1.4230],
        [-1.5950,  0.1992,  0.8661,  ..., -1.9544,  0.7029,  0.7048]],
       grad_fn=<ThAddmmBackward>)
torch.Size([2, 1000])


In [5]:
# we have 2 categorical variables so 1000 -> 500 -> 2
# test the whole process

model = nn.Sequential(
            nn.Linear(1000,500),
            nn.ReLU(),
            nn.BatchNorm1d(500),
            nn.Linear(500,2),
            nn.ReLU()
            )

print(model(resnet(img)))

tensor([[0.1829, 0.0000],
        [0.0000, 1.0460]], grad_fn=<ThresholdBackward0>)


In [6]:
# define loss func & optimizer

loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [7]:
# In order to train with GPU, we need to put the model and variables
# by doing .cuda()

resnet.cuda()
model.cuda()

for i in range(epoch):
    for img,label in img_batch:
        img = Variable(img).cuda()
        label = Variable(label).cuda()

        optimizer.zero_grad()
        output = model(resnet(img))
        loss = loss_func(output,label)
        loss.backward()
        optimizer.step()

    if i % 10 ==0:
        print(loss)

tensor(0.4941, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.0277, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0522, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0409, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0358, device='cuda:0', grad_fn=<NllLossBackward>)


In [9]:
# Check Accuracy of the trained model
# Need to get used to using .cuda() and .data 

model.eval()
correct = 0
total = 0

for img,label in img_batch:
    img = Variable(img).cuda()
    label = Variable(label).cuda()
    
    output = model(resnet(img))
    _, pred = torch.max(output.data,1)
    
    total += label.size(0)
    correct += (pred == label.data).sum()   

print("Accuracy: {}".format(correct/total))

Accuracy: 1
